# Multy-Domain Sentiment Dataset: Sentiment Analisys per domain

## 1. Preprocesamiento
Para el preprocesamiento, se creó la clase "Corpus" que recibe una serie de documentos y obtiene las etiquetas de cada uno, el vocabulario, y las representaciones tf y tfidf.

In [10]:
import os
import pandas as pd
import numpy as np
import math
from scipy.sparse import coo_array

POSITIVE_LABEL = 1
NEGATIVE_LABEL = 0

CATEGORY_PATHS = {
    "books": "../data/books/",
    "dvd": "../data/dvd/",
    "electronics": "../data/electronics/",
    "kitchen": "../data/kitchen/",
}


class Corpus:
    # Atributes
    __vocabulary = {}
    __tf = None
    __tfidf = None
    __documents = None
    __doc_frequencies = None


    def __init__(self, docs: list, vocabulary: dict = None) -> None:
        """
        Creates the vocabulary, the TF and the TFIDF matricees from docs.
        
            Params
            ------
                docs: pd.DataFrame
                    List of documents.
                
                vocabulary: dict | None (default: None)
                    A dictionary where the keys are the terms in the vocabulary
                    and the values are each term's unique id. If set to None,
                    this class will create the vocabulary based on docs.
                    WARNING: if specified, vocabulary must contain a "#UNK#" key
                    for unknown terms.
        """        
        self.__documents = self.__get_docs_df(docs)
        if vocabulary is None:
            self.__vocabulary = self.__load_vocabulary()
        else:
            self.__vocabulary = vocabulary
            
        self.__tf = self.__load_tf()
        self.__tfidf = self.__load_tfidf()

    def __get_docs_df(self, docs: list):
        documents = []
        for doc in docs:
            terms = self.__get_term_counts(doc)
            label = self.__get_label(doc)
            documents.append([terms, label])
        
        return pd.DataFrame(documents, columns=["terms", "label"])

    def __get_term_counts(self, line: str) -> list:
        line_arr = line.split()
        line_arr = line_arr[:-1] # Remove #label#: from the end of array
        terms = {}
        for term in line_arr:
            term_arr = term.split(":")
            terms[term_arr[0]] = int(term_arr[1])
        
        return terms
    
    def __get_label(self, doc: str) -> int:
        doc_arr = doc.split()
        label_str = doc_arr[-1]
        label = label_str.split(":")[-1]
        if label.lower() == "negative":
            return NEGATIVE_LABEL
        elif label.lower() == "positive":
            return POSITIVE_LABEL
    
    def __load_vocabulary(self):
        # Term counts in the whole corpus
        voc = {}
        docs = self.__documents
        for i in range(len(docs)):
            terms_dict = docs.loc[i, "terms"]
            for term in terms_dict:
                if term in voc:
                    voc[term] += terms_dict[term]
                else:
                    voc[term] = terms_dict[term]
        
        # Replace terms with one appearance with UNK
        terms_to_del = []
        for term in voc:
            if voc[term] == 1:
                terms_to_del.append(term)

        # Remove terms that appeare only once
        for term in terms_to_del:
            voc.pop(term)
        
        # Assign unique ids to terms in vocabulary
        bow = {"#UNK#": 0}
        i = 1
        for term in voc:
            bow[term] = i
            i += 1
        
        return bow

    def __load_tf(self) -> np.ndarray:
        voc = self.__vocabulary
        docs = self.__documents
        data = []
        x = []
        y = []
        for i in range(len(docs)):
            doc = docs.iloc[i]
            for term in doc["terms"]:
                if term in voc:
                    data.append(doc["terms"][term])
                    x.append(i)
                    y.append(voc[term])
                else:
                    data.append(doc["terms"][term])
                    x.append(i)
                    y.append(voc["#UNK#"])
        
        sparce_matrix = coo_array((data, (x,y)), shape=(len(docs), len(voc)), dtype=np.uint64).tocsr()
        return sparce_matrix

    def __load_tfidf(self) -> np.ndarray:
        voc = self.__vocabulary
        docs = self.__documents
        tfs = self.__tf
        doc_frec = np.zeros(len(voc))
        # Calculate document frequencies
        for i in range(len(docs)):
            doc = docs.loc[i, "terms"]
            added_unk = False
            for term in doc:
                if term in voc:
                    term_id = voc[term]
                    doc_frec[term_id] += 1
                elif not added_unk:
                    term_id = voc["#UNK#"]
                    doc_frec[term_id] += 1
                    added_unk = True

        total_docs = len(docs)
        data = []
        x = []
        y = []
        for i in range(len(docs)):
            doc = docs.iloc[i]
            unk_tf = 0
            for term in doc["terms"]:
                if term in voc:
                    term_id = voc[term]
                    tf = doc["terms"][term]
                    df = int(doc_frec[term_id])
                    data.append(math.log10(1+tf) * math.log10(total_docs / df))
                    x.append(i)
                    y.append(term_id)
                else:
                    unk_tf += doc["terms"][term]
            
            # Calculate UNK tfidf
            term_id = voc["#UNK#"]
            tf = int(tfs[i, term_id])
            df = int(doc_frec[term_id])
            temp = math.log10(1+tf) * math.log10(total_docs / df)
            if temp < 0:
                raise Exception("HP")
            data.append(math.log10(1+tf) * math.log10(total_docs / df))
            x.append(i)
            y.append(term_id)

        sparce_matrix = coo_array((data, (x, y)), shape=(len(docs), len(voc)), dtype=np.float64).tocsr()        
        return sparce_matrix

    def getVocabulary(self) -> dict:
        return self.__vocabulary
    
    def getDocuments(self) -> pd.DataFrame:
        return self.__documents

    def getTfs(self):
        return self.__tf
    
    def getTfidfs(self):
        return self.__tfidf

# 2. Construcción del modelo y métricas por categoría

In [11]:
def calculate_metrics(labels: list, predicted_labels: list) -> dict:
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(len(labels)):
        act     = labels[i]
        pred    = predicted_labels[i]
        if(act == 1 and pred == 1):
            tp += 1
        elif pred == 1 and act == 0:
            fp += 1
        elif pred == 0 and act == 1:
            fn += 1
        elif pred == 0 and act == 0:
            tn += 1
    
    presition = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 =  2 * presition * recall / (presition + recall)

    return {"presition": presition, "recall": recall, "f1": f1}
    


In [12]:
from sklearn.naive_bayes import MultinomialNB

def test_nv_classifier(X_train: np.ndarray, y_train, X_test: np.ndarray, y_test) -> dict:
    model = MultinomialNB()
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    return calculate_metrics(y_test, y_predicted)

In [13]:
import os

def get_docs_from_folder_paths(folder_paths: list):
    train_docs = []
    test_docs = []
    for path in folder_paths:
        positive_path = os.path.join(path, "positive.review")
        negative_path = os.path.join(path, "negative.review")
        test_path = os.path.join(path, "unlabeled.review")

        f = open(positive_path)
        line = f.readline()
        while line != "":
            train_docs.append(line)
            line = f.readline()
        f.close()

        f = open(negative_path)
        line = f.readline()
        while line != "":
            train_docs.append(line)
            line = f.readline()
        f.close()

        f = open(test_path)
        line = f.readline()
        while line != "":
            test_docs.append(line)
            line = f.readline()
        f.close()

    return train_docs, test_docs

In [14]:
def test_multinomial_nv_by_cat():
    for cat in CATEGORY_PATHS:
        print("=== Multinomial NV for:", cat, "===")
        folder_path = CATEGORY_PATHS[cat]
        train_docs, test_docs = get_docs_from_folder_paths([folder_path])
        train_corpus = Corpus(train_docs)
        test_corpus = Corpus(test_docs, train_corpus.getVocabulary())
        y_train = train_corpus.getDocuments()["label"]
        y_test = test_corpus.getDocuments()["label"]

        # Test with TFs
        print("TFs: ", end="")
        X_train = train_corpus.getTfs()
        X_test = test_corpus.getTfs()
        print(test_nv_classifier(X_train, y_train, X_test, y_test))

        # Test with TFIDFs
        print("TFIDFs: ", end="")
        X_train = train_corpus.getTfidfs()
        X_test = test_corpus.getTfidfs()
        print(test_nv_classifier(X_train, y_train, X_test, y_test))

In [15]:
test_multinomial_nv_by_cat()

=== Multinomial NV for: books ===


KeyboardInterrupt: 

In [29]:
from sklearn.linear_model import LogisticRegression
def test_lr_classifier(X_train: np.ndarray, y_train, X_test: np.ndarray, y_test, \
                        penalty = None, solver = "lbfgs", max_iter = 100) -> dict:
    model = LogisticRegression(penalty=penalty, solver=solver, max_iter=max_iter)
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    return calculate_metrics(y_test, y_predicted)

In [19]:
def test_lr_classifier_by_cat():
    for cat in CATEGORY_PATHS:
        print("=== LR for:", cat, "===")
        folder_path = CATEGORY_PATHS[cat]
        train_docs, test_docs = get_docs_from_folder_paths([folder_path])
        train_corpus = Corpus(train_docs)
        test_corpus = Corpus(test_docs, train_corpus.getVocabulary())
        y_train = train_corpus.getDocuments()["label"]
        y_test = test_corpus.getDocuments()["label"]

        # Test with TFs
        print("TFs: ", end="")
        X_train = train_corpus.getTfs().toarray()
        X_test = test_corpus.getTfs()
        print(test_lr_classifier(X_train, y_train, X_test, y_test, "l1", "liblinear"))

        # Test with TFIDFs
        print("TFIDFs: ", end="")
        X_train = train_corpus.getTfidfs().toarray()
        X_test = test_corpus.getTfidfs()
        print(test_lr_classifier(X_train, y_train, X_test, y_test, "l1", "liblinear"))
        

In [20]:
test_lr_classifier_by_cat()

=== LR for: books ===
TFs: {'presition': 0.8010182435299109, 'recall': 0.833922261484099, 'f1': 0.817139147370699}
TFIDFs: {'presition': 0.8291479820627803, 'recall': 0.816696113074205, 'f1': 0.8228749443702715}
=== LR for: dvd ===
TFs: {'presition': 0.7784522003034902, 'recall': 0.8516878804648589, 'f1': 0.8134249471458774}
TFIDFs: {'presition': 0.8061116965226555, 'recall': 0.8467072495849475, 'f1': 0.8259109311740892}
=== LR for: electronics ===
TFs: {'presition': 0.8460754332313966, 'recall': 0.871543577178859, 'f1': 0.8586206896551724}
TFIDFs: {'presition': 0.8542094455852156, 'recall': 0.8736436821841093, 'f1': 0.8638172694237757}
=== LR for: kitchen ===
TFs: {'presition': 0.8580923894981721, 'recall': 0.8740690589031821, 'f1': 0.8660070434345127}
TFIDFs: {'presition': 0.8710990502035278, 'recall': 0.8693297224102912, 'f1': 0.8702134869535751}


## 3. Construcción del modelo y métricas con un compilado de todas las categorías

In [25]:
def test_nv_classifier_complete():
    print("=== TEST NV FOR FULL CORPUS ===")
    folder_paths = []
    for cat in CATEGORY_PATHS:
        folder_paths.append(CATEGORY_PATHS[cat])

    train_docs, test_docs = get_docs_from_folder_paths(folder_paths)
    train_corpus = Corpus(train_docs)
    test_corpus = Corpus(test_docs, train_corpus.getVocabulary())
    y_train = train_corpus.getDocuments()["label"]
    y_test = test_corpus.getDocuments()["label"]

    # Test with TFs
    print("TFs: ", end="")
    X_train = train_corpus.getTfs()
    X_test = test_corpus.getTfs()
    print(test_nv_classifier(X_train, y_train, X_test, y_test))

    # Test with TFIDFs
    print("TFIDFs: ", end="")
    X_train = train_corpus.getTfidfs()
    X_test = test_corpus.getTfidfs()
    print(test_nv_classifier(X_train, y_train, X_test, y_test))

In [26]:
test_nv_classifier_complete()

=== TEST NV FOR FULL CORPUS ===
TFs: {'presition': 0.8453076323987538, 'recall': 0.8786682857721109, 'f1': 0.8616651781284111}
TFIDFs: {'presition': 0.849905444411267, 'recall': 0.8640963367739324, 'f1': 0.8569421446133775}


In [32]:
def test_lr_classifier_complete():
    print("=== TEST NV FOR FULL CORPUS ===")
    folder_paths = []
    for cat in CATEGORY_PATHS:
        folder_paths.append(CATEGORY_PATHS[cat])

    train_docs, test_docs = get_docs_from_folder_paths(folder_paths)
    train_corpus = Corpus(train_docs)
    test_corpus = Corpus(test_docs, train_corpus.getVocabulary())
    y_train = train_corpus.getDocuments()["label"]
    y_test = test_corpus.getDocuments()["label"]

    # Test with TFs
    print("TFs: ", end="")
    X_train = train_corpus.getTfs()
    X_test = test_corpus.getTfs()
    print(test_lr_classifier(X_train, y_train, X_test, y_test, max_iter=4000))

    # Test with TFIDFs
    print("TFIDFs: ", end="")
    X_train = train_corpus.getTfidfs()
    X_test = test_corpus.getTfidfs()
    print(test_lr_classifier(X_train, y_train, X_test, y_test))

In [33]:
test_lr_classifier_complete()

=== TEST NV FOR FULL CORPUS ===
TFs: {'presition': 0.8593610032845626, 'recall': 0.8737097753491196, 'f1': 0.8664759897636611}
TFIDFs: {'presition': 0.8904557531166973, 'recall': 0.8818053025703299, 'f1': 0.8861094163107586}
